<a href="https://colab.research.google.com/github/Levan-Danelia/FRTB/blob/main/FRTB_CRDL_Securitization_ACTP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Cell 1: Imports, Helper Function, and Initial Data Setup
import pandas as pd
import numpy as np

def calculate_capital_charge(weighted_sensitivities, correlation):
    """
    Calculates the intra-bucket capital charge (Kb) based on weighted sensitivities and a correlation matrix.
    This function implements the formula from Article 325f(7).

    Args:
        weighted_sensitivities (list): A list of weighted sensitivity values.
        correlation (float): The single correlation value to use for all off-diagonal elements.

    Returns:
        float: The calculated intra-bucket capital charge.
    """
    ws = np.array(weighted_sensitivities)

    # Create a correlation matrix
    num_factors = len(ws)
    corr_matrix = np.full((num_factors, num_factors), correlation)
    np.fill_diagonal(corr_matrix, 1)

    # Calculate the sum of squared weighted sensitivities
    sum_sq_ws = np.sum(ws**2)

    # Calculate the sum of cross-products
    sum_cross_products = 0
    for i in range(num_factors):
        for j in range(num_factors):
            if i != j:
                sum_cross_products += corr_matrix[i, j] * ws[i] * ws[j]

    # The value inside the square root, floored at zero
    inner_value = max(0, sum_sq_ws + sum_cross_products)

    # Calculate Kb
    kb = np.sqrt(inner_value)
    return kb

# --- Initial Portfolio Data ---
# Define the portfolio based on the provided data.
portfolio_data = [
    {'Position': 1, 'Bucket': 13, 'Curve': 'CDS', 'Sector': 'Consumer', 'Tenor': '6M', 'Issuer': 'Issuer 1', 'Sensitivity': 2344},
    {'Position': 2, 'Bucket': 13, 'Curve': 'CDS', 'Sector': 'Consumer', 'Tenor': '1Y', 'Issuer': 'Issuer 1', 'Sensitivity': -1812715},
    {'Position': 3, 'Bucket': 13, 'Curve': 'CDS', 'Sector': 'Consumer', 'Tenor': '3Y', 'Issuer': 'Issuer 1', 'Sensitivity': -5563010}
]
df = pd.DataFrame(portfolio_data)

print("--- FRTB ASA: CSR Securitization ACTP Delta Calculation ---")
print("Cell 1 executed: Libraries imported, helper function defined, and data loaded.")
print("-" * 60)

--- FRTB ASA: CSR Securitization ACTP Delta Calculation ---
Cell 1 executed: Libraries imported, helper function defined, and data loaded.
------------------------------------------------------------


In [3]:
# Cell 2: Step 1 - Definition of Risk Factors
print("\n### Step 1: Definition of Risk Factors ###")
df['Risk Factor (Issuer, Tenor)'] = df['Issuer'] + ', ' + df['Tenor']
print("Risk factors are defined by the underlying issuer and tenor.")
print(df[['Position', 'Bucket', 'Risk Factor (Issuer, Tenor)']].to_string(index=False))
print("-" * 60)


### Step 1: Definition of Risk Factors ###
Risk factors are defined by the underlying issuer and tenor.
 Position  Bucket Risk Factor (Issuer, Tenor)
        1      13                Issuer 1, 6M
        2      13                Issuer 1, 1Y
        3      13                Issuer 1, 3Y
------------------------------------------------------------


In [4]:
# Cell 3: Step 2 - Definition of Delta Sensitivities
print("\n### Step 2: Definition of Delta Sensitivities ###")
print("Initial sensitivities are provided for each position.")
print(df[['Position', 'Risk Factor (Issuer, Tenor)', 'Sensitivity']].to_string(index=False))
print("-" * 60)


### Step 2: Definition of Delta Sensitivities ###
Initial sensitivities are provided for each position.
 Position Risk Factor (Issuer, Tenor)  Sensitivity
        1                Issuer 1, 6M         2344
        2                Issuer 1, 1Y     -1812715
        3                Issuer 1, 3Y     -5563010
------------------------------------------------------------


In [5]:
# Cell 4: Step 3 - Net Sensitivities
print("\n### Step 3: Net Sensitivities ###")
df['Net Sensitivity'] = df['Sensitivity']
print("No netting is applicable as each position has a unique risk factor (tenor).")
print(df[['Position', 'Risk Factor (Issuer, Tenor)', 'Net Sensitivity']].to_string(index=False))
print("-" * 60)


### Step 3: Net Sensitivities ###
No netting is applicable as each position has a unique risk factor (tenor).
 Position Risk Factor (Issuer, Tenor)  Net Sensitivity
        1                Issuer 1, 6M             2344
        2                Issuer 1, 1Y         -1812715
        3                Issuer 1, 3Y         -5563010
------------------------------------------------------------


In [6]:
# Cell 5: Step 4 - Risk Weights and Weighted Sensitivities
print("\n### Step 4: Risk Weights and Weighted Sensitivities ###")
risk_weight = 0.12
df['Risk Weight'] = f"{risk_weight:.1%}"
df['Weighted Sensitivity'] = df['Net Sensitivity'] * risk_weight
print(f"Applying a risk weight of {risk_weight:.1%} for Bucket 13 (Article 325ak).")
print(df[['Position', 'Risk Factor (Issuer, Tenor)', 'Net Sensitivity', 'Risk Weight', 'Weighted Sensitivity']].to_string(index=False))
sb = df['Weighted Sensitivity'].sum()
print(f"\nSum of Weighted Sensitivities (Sb) for Bucket 13: {sb:,.0f}")
print("-" * 60)


### Step 4: Risk Weights and Weighted Sensitivities ###
Applying a risk weight of 12.0% for Bucket 13 (Article 325ak).
 Position Risk Factor (Issuer, Tenor)  Net Sensitivity Risk Weight  Weighted Sensitivity
        1                Issuer 1, 6M             2344       12.0%                281.28
        2                Issuer 1, 1Y         -1812715       12.0%            -217525.80
        3                Issuer 1, 3Y         -5563010       12.0%            -667561.20

Sum of Weighted Sensitivities (Sb) for Bucket 13: -884,806
------------------------------------------------------------


In [7]:
# Cell 6: Step 5 & 6 - Determine Correlation Coefficients (Medium Scenario)
print("\n### Step 5 & 6: Determine Correlation Coefficients (Medium Scenario) ###")
corr_medium = 1.0 * 0.65 * 1.0
print(f"The intra-bucket correlation for the medium scenario is: {corr_medium:.4f}")
print("Cross-bucket correlation is not applicable as all positions are in a single bucket.")
print("-" * 60)


### Step 5 & 6: Determine Correlation Coefficients (Medium Scenario) ###
The intra-bucket correlation for the medium scenario is: 0.6500
Cross-bucket correlation is not applicable as all positions are in a single bucket.
------------------------------------------------------------


In [8]:
# Cell 7: Step 7 & 8 - Aggregation (Medium Scenario)
print("\n### Step 7 & 8: Aggregation (Medium Scenario) ###")
weighted_sensitivities = df['Weighted Sensitivity'].tolist()
kb_medium = calculate_capital_charge(weighted_sensitivities, corr_medium)
print(f"The intra-bucket capital charge for Bucket 13 (Kb) is: £{kb_medium:,.0f}")
k_risk_class_medium = kb_medium
print(f"Total Delta Capital (Medium Scenario): £{k_risk_class_medium:,.0f}")
print("-" * 60)


### Step 7 & 8: Aggregation (Medium Scenario) ###
The intra-bucket capital charge for Bucket 13 (Kb) is: £825,474
Total Delta Capital (Medium Scenario): £825,474
------------------------------------------------------------


In [9]:
# Cell 8: Step 9 - Correlation Scenarios
print("\n### Step 9: Correlation Scenarios ###")
corr_high = min(corr_medium * 1.25, 1.0)
kb_high = calculate_capital_charge(weighted_sensitivities, corr_high)
corr_low = max(2 * corr_medium - 1, 0.75 * corr_medium)
kb_low = calculate_capital_charge(weighted_sensitivities, corr_low)

scenario_results = {
    'Scenario': ['Low', 'Medium', 'High'],
    'Intra-Bucket Correlation': [f"{corr_low:.4f}", f"{corr_medium:.4f}", f"{corr_high:.4f}"],
    'Delta Capital Charge (£)': [f"{kb_low:,.0f}", f"{kb_medium:,.0f}", f"{kb_high:,.0f}"]
}
df_scenarios = pd.DataFrame(scenario_results)
print("Recalculating the capital charge under High and Low correlation scenarios.")
print(df_scenarios.to_string(index=False))
print("-" * 60)


### Step 9: Correlation Scenarios ###
Recalculating the capital charge under High and Low correlation scenarios.
Scenario Intra-Bucket Correlation Delta Capital Charge (£)
     Low                   0.4875                  796,426
  Medium                   0.6500                  825,474
    High                   0.8125                  853,534
------------------------------------------------------------


In [10]:
# Cell 9: Step 10 - Final Charge Calculation
print("\n### Step 10: Final Charge Calculation ###")
final_charge = max(kb_low, kb_medium, kb_high)
print("The final delta charge is the maximum of the three scenarios (Article 325h(4)).")
print(f"\nFinal CSR ACTP Delta Capital Requirement: £{final_charge:,.0f}")
print("=" * 60)


### Step 10: Final Charge Calculation ###
The final delta charge is the maximum of the three scenarios (Article 325h(4)).

Final CSR ACTP Delta Capital Requirement: £853,534
